In [5]:
import os
import glob
import json
import xml.etree.ElementTree as ET
import json
import string

In [ ]:
# 读取mesh词汇
tree = ET.parse("mesh/desc2022.xml")
root = tree.getroot()
animals = []
infections = []
respi = []
respiratory = []
ages = []
immunes = []
neoplasms = []
hemics = []
for i in root:

    if i.find("TreeNumberList/TreeNumber") is not None:

        if "B01.050" in i.find("TreeNumberList/TreeNumber").text:
            
            animals.append(i.find("DescriptorName/String").text)
        
        if "C01" in i.find("TreeNumberList/TreeNumber").text:
            
            infections.append(i.find("DescriptorName/String").text)

        if "C08" in i.find("TreeNumberList/TreeNumber").text:
            
            respiratory.append(i.find("DescriptorName/String").text)
        
        if "M01.060" in i.find("TreeNumberList/TreeNumber").text:
            
            ages.append(i.find("DescriptorName/String").text)
        
        if "A04" in i.find("TreeNumberList/TreeNumber").text:
            
            respi.append(i.find("DescriptorName/String").text)
        
        if "C20" in i.find("TreeNumberList/TreeNumber").text:
            
            immunes.append(i.find("DescriptorName/String").text)
        
        if "C04" in i.find("TreeNumberList/TreeNumber").text:
            
            neoplasms.append(i.find("DescriptorName/String").text)
        
        if "C15" in i.find("TreeNumberList/TreeNumber").text:
            
            hemics.append(i.find("DescriptorName/String").text)
        
animals.remove("Humans")

# 去除动物文献

In [2]:
import json
with open("search_results/search_results.txt", "r") as f:
    content = f.read()
    spec_papers_dic =  json.loads(content)

In [3]:
#统计
for spec_type in spec_papers_dic:
    spec_num = 0
    paper_num = 0
    for k,v in spec_papers_dic[spec_type].items():
        if len(v) > 0:
            spec_num = spec_num + 1
            paper_num = paper_num + len(v)
    print(spec_num, paper_num)

1602 2994859
703 579919
1094 1842629


In [11]:
# 去除动物文献
animals_set = set(animals)
for spec_type in spec_papers_dic:
    for k, v in spec_papers_dic[spec_type].items():
        vbp = {}
        for paper_id, paper in v.items():
            if len(set(sum(paper["Meshhead"],[])) & animals_set) > 0:
                if "Humans" in set(sum(paper["Meshhead"],[])):
                    vbp[paper_id] = paper
            else:
                vbp[paper_id] = paper
        spec_papers_dic[spec_type][k] = vbp

In [4]:
with open("search_results/search_results_hanimals.txt", "w") as f:
    content = json.dumps(spec_papers_dic)
    f.write(content)

# 筛选肺炎文献

In [2]:
pneumonia = {"bronchitis","bronchopneumonia",
             "pneumonia","pneumonitis","pneumonic",
             "pleuropneumonia","pleural"}
LungRelated = {"bronchus","bronchial",
               "tracheobronchial","bronchiolar","bronchopulmonary",
               "bronchoalveolar","bronchoscopy",
               "lung","pulmonary","pneumonic","bronchopulmonary","respiratory","respiration"}
infection = {"infection","infections","abscess",
             "pathology","etiology","diseases",
             "bacteremia","sepsis","septic","pathogenicity","legionellosis","empyema","necrosis"}

In [3]:
with open("search_results/search_results_hanimals.txt","r") as f:
    spec_dict = json.loads(f.read())

In [7]:
pats = {'Bactology':{}, 'Mycology':{}, 'Virology':{}}

for sped, sped_paper in spec_dict.items():
    
    for spec, spec_paper in sped_paper.items():
        
        vbp = {}
        for paper_id, paper in spec_paper.items():
            
            spec = spec.lower()
            text = paper["texts"]["Ab"] +" "+" ".join(sum(paper["Meshhead"],[])) # 添加mesh词汇
            text = text.translate(str.maketrans(string.punctuation, " "*32,)).replace("  ", " ")
            text = text.lower()
            words = set(text.split(" "))
            if len(words & pneumonia) > 0:
                # if "colonization" not in words:
                #     if "%s / isolation & purification" % spec not in " / ".join(sum(paper["Meshhead"],[])).lower():
                vbp[paper_id] = paper

            else:
                if len(words & LungRelated) > 0:
                    if len(words & infection) > 0:
                        # if "colonization" not in words:
                        #     if "%s / isolation & purification" % spec not in " / ".join(sum(paper["Meshhead"],[])).lower():
                        vbp[paper_id] = paper
        if len(vbp) > 0:
            pats[sped][spec] = vbp

In [9]:
with open("search_results/search_results_pneumonia.txt", "w") as f:
    content = json.dumps(pats)
    f.write(content)